In [1]:
import os

def get_all_file_in_folder(folder):
    for _, _, files in os.walk(folder):
        filenames = list(files)
        file_paths = [os.path.join(folder, f) for f in filenames]
        return filenames, file_paths
    return [], []

In [21]:
import pandas as pd

df = pd.read_csv("../data/context/merged_tokens.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

In [22]:
# df[(df["score"] == 3 )|(df["count"]>1)]
df.dropna(inplace=True)
df = df[(df["score"] == 3 )|(df["count"]>10)]
# df = df.sort_values(by="token", key=lambda x: -x.str.len())
df = df.sort_values(by="count", key=lambda x: -x)
df = df[df["count"]<100000]

In [24]:
df.shape
df.insert(0, 'id', range(len(df)))
dict_df = df

In [25]:
# df[df["token"] == "123"].iloc[0]
dict_df

,id,token,score,count
16,0,nam,3,80269
17,1,thành,3,73905
18,2,tiếng,3,70842
19,3,diện tích,3,69009
20,4,bắc,3,68083
...,...,...,...,...
1831834,3001541,lẫm khưu,3,1
1831835,3001542,cao trương phản công,3,1
1831836,3001543,lê di vua,3,1
1831830,3001544,tcn tề quốc,3,1


In [ ]:
batch = 100
size = 1273500 #1273469
token_size = dict_df.shape[0]
expect_file_rows = 100000


import scipy.sparse as sp
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import pickle
import os
import time

# save dict to file
print("Save dictionary\n")
# dict_df.to_csv("dictionary.csv")

# tokens = {}
# for index, row in dict_df.iterrows():
#     tokens[row['token']] = (row['id'], row['score'])

# map context id to matrix id
map_context_id_to_matrix_id = {}

file_partials = ["aa", "ab", "ac", "ad", "ae", "af", "ag", "ah", "ai", "aj", "ak", "al", "am"]

# _, filepaths = get_all_file_in_folder("../data/context/.token")
filepaths = [
    os.path.join("../data/context/.token", "wikipedia_%s.processed" % f)
    for f in file_partials
]
# filepaths = filepaths[:2]

# idx = 0
# for fn in tqdm(filepaths):
#     with open(fn, "r") as f:
#         for l in f:
#             jdata = json.loads(l)
#             map_context_id_to_matrix_id[jdata["id"]] = idx
#             idx += 1
            
# map_matrix_id_to_context_id = {v:k for k,v in map_context_id_to_matrix_id.items()}

# save map
# with open('map_context_id_to_matrix_id.pickle', 'wb') as handle:
#     pickle.dump(map_context_id_to_matrix_id, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('map_matrix_id_to_context_id.pickle', 'wb') as handle:
#     pickle.dump(map_matrix_id_to_context_id, handle, protocol=pickle.HIGHEST_PROTOCOL)


# build sparse matrix
print("Build sparse matrix")
matrix_id = 0
indexer = None

for fn in filepaths:# tqdm(filepaths):
    t_point = time.time()
    b = 0
    indptr = [0]
    indices = []
    data = []
    with open(fn, "r") as f:
        print("Process file %s\n" % fn)
#         c_ = 0
        for _ in tqdm(range(expect_file_rows)):
            l = f.readline()
#         for l in f:
#             print('new')
#             c_ += 1
#             if c_ > 5:
#                 break
            jdata = json.loads(l)
#             dt = time.time()
            dict_token = dict_df[dict_df["token"].isin(jdata["tokens"])]
            p_ = False
            for _, r in dict_token.iterrows():
                p_ = True
                data.append(r.score)
                indices.append(r.id)
#             print(time.time() -dt)
            if not p_:
                continue
            indptr.append(len(indices))

#             print("id done %s, matrix id %s" % (jdata["id"], matrix_id))
            map_context_id_to_matrix_id[jdata["id"]] = matrix_id
            b += 1
            matrix_id += 1
            if b == batch:
                if indexer is None:
                    indexer = sp.csr_matrix((data, indices, indptr), dtype=np.int8, shape=(len(indptr)-1,token_size))
#                     print('done')
                else:
                    tmp = sp.csr_matrix((data, indices, indptr), dtype=np.int8, shape=(len(indptr)-1,token_size))
                    indexer = sp.vstack((indexer, tmp), format='csr')
#                     print('done')
                indptr = [0]
                indices = []
                data = []
                b = 0
#                 print("done batch, matrix id %s, time %s\n" % (matrix_id, time.time()-t_point))
    if b > 0:
        if indexer is None:
            indexer = sp.csr_matrix((data, indices, indptr), dtype=np.int8, shape=(len(indptr)-1,token_size))
#             print(indexer.shape)
        else:
            tmp = sp.csr_matrix((data, indices, indptr), dtype=np.int8, shape=(len(indptr)-1,token_size))
#             print(tmp.shape, indexer.shape)
            indexer = sp.vstack((indexer, tmp), format='csr')
        print("done batch, matrix id %s, time %s\n" % (matrix_id, time.time()-t_point))
    print("done file, matrix id %s, time %s\n" % (matrix_id, time.time()-t_point))
            
    print("Save checkpoint\n")
        
    with open('map_context_id_to_matrix_id.pickle', 'wb') as handle:
        pickle.dump(map_context_id_to_matrix_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # save to file
    print("Save indexer to file\n")
    sp.save_npz("indexer.npz", indexer, compressed=True)
            

Save dictionary

Build sparse matrix
Process file ../data/context/.token/wikipedia_aa.processed



  2%|█▏                                                                      | 1657/100000 [05:48<5:48:25,  4.70it/s]